In [4]:
pip install request

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request


In [5]:
pip install beautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in c:\users\ksy\anaconda3\lib\site-packages (from requests->webdriver-manager) (1.25.9)



In [8]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import re
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen
from selenium.webdriver import ActionChains

driver = webdriver.Chrome('C:/Users/KSY/Desktop/chromedriver.exe')
driver.get("https://www.idus.com/oc") #tab[0]
driver.execute_script('window.open("about:blank", "_blank");') #창 1개더 열어놓기 tab[1]
tabs = driver.window_handles
time.sleep(3)

def scroll():
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(1)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

with open ("ourdus_class.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite)
    time.sleep(3)
    driver.switch_to_window(tabs[0])
    scroll()
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    myWriter.writerow(["카테고리", "이름","배너 사진 URL", "할인율", "가격/월",'전체가격','상세페이지 URL','수강시작일','수강기간','난이도','준비물','클래스소개','클래스댓글'])
       
    classlist = soup.findAll('li',{'class':'ui_grid__item'}) #모든 클래스 목록
    count = 0
    
    data=[] 
    
    #클래스 한개씩 작업
    for row in classlist:   
        #img_URL얻으려고 전처리과정
        img = row.find('div',{'class':'thumb-img'})
        img = img["style"]
        start=img.find('\"')
        end = img.find('\"',start+1)
        img_URL = img[start+1:end]

        #img_URL로 로컬 저장소에 사진 저장
        with urlopen(img_URL) as f:
            count +=1
            with open('./Ourdus/class_img/'+ str(count)+ '.jpg',"wb") as h:
                img=f.read()
                h.write(img)
        
        
        label = row.find('div',{'class':'label'}).text #카테고리
        name = row.find('div',{'class':'class-name'}).text #이름
        
        # 할인률 없는것들도 있어서 try~catch 사용
        try:
            discount = row.find('span',{'class':'sale-rate'}).text.replace('%','')
            discount=int(discount)
        except:
            discount=0
            
        #가격 전처리 (/월 가격임)
        price = row.find('span',{'class':'price'}).text.replace('\n','').replace(',','')
        price = int(price)
        
        data.append([label,name,img_URL,discount,price])
         
        # 현재 클래스의 상세 페이지 URL
        info_URL = row.find({'a':'data-v-27ea337a'})
        info_URL = info_URL["href"]
        info_URL = "https://www.idus.com/" + str(info_URL)
        
        
        driver.switch_to_window(tabs[1]) #두번째 탭으로 이동
        driver.get(info_URL)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        
        basic_info = soup.findAll({'dd':'data-v-af0d8610'}) #수강시작일 + 수강기간 + 난이도 + 준비물 다 긁어옴
        info=[]
        for i in basic_info: #한개씩 저장
            info.append(i.text)
        
        start_date=info[0] #수강시작일
        duration = info[1] #수강기간
        difficulty=info[2] #난이도
        material = info[3] #준비물
        
        #총 가격 전처리 (/월 말고)
        total_price = soup.find('p',{'class':'option-text price'}).text
        total_price=total_price.replace('원','').replace(',','')
        total_price=int(total_price)
        
        #클래스 정보 소개
        article = soup.findAll('p',{'class':'article_block__para bot_spacing'})
        article_info=[]
        for i in article:
            article_info.append(i.text)
        
        #클래스 댓글
        comment = soup.findAll('div',{'class':'area-txt'})
        comment_info=[]
        for i in comment:
            try:
                user_name=i.find({'span':'data-v-698f8bea'}).text
                user_comment=i.find('p',{'class':'txt'}).text
                comment_info.append([user_name,user_comment])   # [작성자 이름, 코멘트 내용] 이 한묶음
            except:
                continue

        tmp = data[count-1]+[total_price,info_URL,start_date,duration,difficulty,material,article_info,comment_info]
        
        myWriter.writerow(tmp)
        print(tmp)
        driver.switch_to_window(tabs[0])
    
 

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 86
Current browser version is 88.0.4324.104 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


In [32]:
import time
import re
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen
from selenium.webdriver import ActionChains

driver = webdriver.Chrome('C:/Users/KSY/Desktop/chromedriver.exe')


 
tabs = driver.window_handles
driver.switch_to_window(tabs[0])
driver.get("https://www.idus.com/oc")

time.sleep(3)

html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
classlist = soup.findAll('li',{'class':'ui_grid__item'})

for row in classlist:
        info_URL = row.find({'a':'data-v-27ea337a'})
        info_URL = info_URL["href"]
        info_URL = "https://www.idus.com/" + str(info_URL)
        print(info_URL)
        driver.switch_to_window(tabs[1])
        driver.get(info_URL)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        basic_info = soup.findAll({'dd':'data-v-af0d8610'})
        info=[]
        for i in basic_info:
            info.append(i.text)
        
        print(info)
        start_date=info[0]
        duration = info[1]
        difficulty=info[2]
        material = info[3]
        total_price = soup.find('p',{'class':'option-text price'}).text
        total_price=total_price.replace('원','')
        total_price=int(total_price)
        print(total_price)
        article = soup.findAll('p',{'class':'article_block__para bot_spacing'})
        article_info=[]
        for i in article:
            article_info.append(i.text)
        print(article_info)
        
        comment = soup.findAll('div',{'class':'area-txt'})
        comment_info=[]
        for i in comment:
            try:
                user_name=i.find({'span':'data-v-698f8bea'}).text
                user_comment=i.find('p',{'class':'txt'}).text
                comment_info.append([user_name,user_comment])
            except:
                continue
        print(comment_info)
            
        



<ipython-input-32-ef7a2cd331d4>:15: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[0])


https://www.idus.com//oc/class/140


IndexError: list index out of range